# softmax回归-从零实现

In [20]:
import torch
import torchvision
import numpy as np
import sys
sys.path.append('./d2lzh/')
import d2lzh_pytorch as d2l

### 1.获取数据

In [21]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)


print(type(train_iter), type(test_iter))

<class 'torch.utils.data.dataloader.DataLoader'> <class 'torch.utils.data.dataloader.DataLoader'>


### 2.模型参数初始化

In [22]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)

In [23]:
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True) 

print(W.is_leaf, b.is_leaf)

True True


### 3.Softmax运算

In [24]:
# X = torch.tensor([[1, 2, 3], [4, 5, 6]])
# print(X.sum(dim=0, keepdim=True))  # dim=0, 按照列求和
# print(X.sum(dim=1, keepdim=True))  # dim=1, 按照行求和

In [25]:
# 假设每行对应于一个输入样本，每列对应于他的预测概率。那么需要用softmax函数
# 对每行元素进行 先指数运算，接着做相加运算，后相除运算。
# > 结果，对每个随机矩阵，每行值的和为1,且每个元素非负。

def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp / partition  # 这里应用了广播机制

In [26]:
# X = torch.rand((2, 5))
# X_prob = softmax(X)
# print(X_prob, X_prob.sum(dim=1))

### 4.定义模型

In [27]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)

### 5.定义损失函数

In [28]:
# # gather函数的使用
# y_hat = torch.Tensor([[0.1, 0.3, 0.6],  # 两个样本，三个类
#                       [0.3, 0.2, 0.5]]) # 每类（经过softmax处理后）的预测概率
# y = torch.LongTensor([0, 2])     # 两个样本的实际类别（第零类，第二类）
# dim = 1                          # 列方向dim=0, 行方向dim=1
# y_view = y.view(-1, 1)
# y_hat.gather(dim, y_view)        # 索引y_hat在每个y_view上dim方向的值（看结果）

In [29]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat.gather(1, y.view(-1, 1)))

### 6.计算分类准确率

In [30]:
# 分类准确率即，正确预测数与总预测数之比
# 如何确定正确预测数：给定某个样本的预测概率分布y_hat, 把概率最大值作为预测类别
# 如果这个类别与真实类别一致，则为正确预测数

# 应用到单个样本的处理过程

def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

# (y_hat.argmax(dim=1) == y) 的返回类型为ByteTensor
# 用 float()函数将其转换为0/1的浮点型 Tensor

In [31]:
# print(accuracy(y_hat, y))   

In [32]:
# 应用到包含多个样本的Fashion-MNIST数据集上
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中描述

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [33]:
print(evaluate_accuracy(test_iter, net))

0.0607


### 7.训练模型

In [34]:

# num_epochs, lr = 5, 0.1

# def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, 
#              params=None, lr=None, optimizer=None):
#     for epoch in range(num_epochs):
#         train_loss_sum, train_acc_sum, n_examples = 0.0, 0.0, 0
#         for X, y in train_iter:
#             y_hat = net(X)
#             ls = loss(y_hat, y).sum()

#             print('Before')
#             print(params[0].is_leaf, params[1].is_leaf)
            
#             # 梯度清零
            
#             if optimizer is not None:
#                 optimizer.zero_grad()
#             elif params is not None and params[0].grad is not None:
#                 for param in params:
#                     param.grad.data.zero_()
                    
#             print('After')
#             print(params[0].is_leaf, params[1].is_leaf)
                    
#             ls.backward()   # 这一步可以计算得到各个grad的更新值
#             if optimizer is None:
#                 d2l.sgd(params, lr, batch_size)
#             else:
#                 optimizer.step()   # # “softmax回归的简洁实现”一节将用到
                
#             train_loss_sum += ls.item()  # 每遍历完一个batch就要将loss汇总起来
#             train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
#             n_examples += y.shape[0]
            
#         test_acc = evaluate_accuracy(test_iter, net)
#         print("Epoch %d, loss %.4f, train accuracy %.3f, test accuracy %.3f" 
#              % (epoch + 1, train_loss_sum / n_examples, train_acc_sum / n_examples, test_acc))
                    

In [35]:
num_epochs, lr = 5, 0.1

# 本函数已保存在d2lzh包中方便以后使用
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到
            
            
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

RuntimeError: leaf variable has been moved into the graph interior

In [19]:
# print(torch.__version__)
# print(torchvision.__version__)

1.3.1
0.4.2
